# Hierarchical Planning



## Setup

We start by downloading (from github) the unified planning library.

In [ ]:
%pip install unified-planning

# A *scheduling* primer

`unified-planning` provides initial support for modeling scheduling problems:

 - reuse state definition (`Type`,`Fluent`, initial state, (timed) goals, ...)
 - replace `Action` with `Activity`
 - add syntactic sugar for common patterns in scheduling problems
 
 
 
 

In [2]:
from unified_planning.shortcuts import *
from unified_planning.model.scheduling import SchedulingProblem

# Create an empty problem called factory
problem = SchedulingProblem("factory")
problem

problem name = factory

fluents = [
]

initial fluents default = [
]

initial values = [
]


BASE: {
  }

Activities:
  

## Resources as numeric fluents

A `SchedulingProblem` allows boolean, symbolic and numeric fluents to state representation, just like a regular planning problem.

In addition, it exposes an [`add_resource`](https://unified-planning.readthedocs.io/en/latest/api/model/scheduling/SchedulingProblem.html#unified_planning.model.scheduling.SchedulingProblem.add_resource) method that eases the definition of reusable resources:

In [3]:
# Create two unary resources, one for each machine
machine1 = problem.add_resource("machine1", capacity=1)
machine2 = problem.add_resource("machine2", capacity=1)
# Create a resource with capacity 4 representing the available operators
operators = problem.add_resource("operators", capacity=4)

print(problem)

problem name = factory

fluents = [
  integer[0, 1] machine1
  integer[0, 1] machine2
  integer[0, 4] operators
]

initial fluents default = [
  integer[0, 1] machine1 := 1
  integer[0, 1] machine2 := 1
  integer[0, 4] operators := 4
]

initial values = [
]


BASE: {
  }

Activities:
  


## Activities

An [`Activity`](https://unified-planning.readthedocs.io/en/latest/api/model/scheduling/Activity.html) is essentially a **durative action** present **exactly once** in the solution.

In [4]:
# Create an activity a1 that has a duration of 3 time units that uses the machine 1
a1 = problem.add_activity("a1", duration=3)
a1.uses(machine1, amount=1)

print(a1)

a1 {
    duration = [3, 3]
    effects = [
      start(a1):
        machine1 -= 1:
      end(a1):
        machine1 += 1:
    ]
  }


In [5]:
# Specify that activity a1 requires 2 operators to be executed
a1.uses(operators, amount=2)
print(a1)

a1 {
    duration = [3, 3]
    effects = [
      start(a1):
        machine1 -= 1:
        operators -= 2:
      end(a1):
        machine1 += 1:
        operators += 2:
    ]
  }


In [6]:
# Create a new activity a2 that lasts 6 time units and require machine2 and 1 operator
a2 = problem.add_activity("a2", duration=6)
a2.uses(operators)  # default usage is 1
a2.uses(machine2)

# Require that activity be finished by time unit 14
a2.add_deadline(14)
a2

a2 {
    duration = [6, 6]
    constraints = [
      (end(a2) <= 14)
    ]
    effects = [
      start(a2):
        operators -= 1:
        machine2 -= 1:
      end(a2):
        operators += 1:
        machine2 += 1:
    ]
  }

In [7]:
# finish a2 before starting a1
problem.add_constraint(LT(a2.end, a1.start))

# One worker is unavailable over [17, 25)
problem.add_decrease_effect(17, operators, 1)
problem.add_increase_effect(25, operators, 1)
problem

problem name = factory

fluents = [
  integer[0, 1] machine1
  integer[0, 1] machine2
  integer[0, 4] operators
]

initial fluents default = [
  integer[0, 1] machine1 := 1
  integer[0, 1] machine2 := 1
  integer[0, 4] operators := 4
]

initial values = [
]


BASE: {
    constraints = [
      (end(a2) < start(a1))
    ]
    effects = [
      start + 17:
        operators -= 1:
      start + 25:
        operators += 1:
    ]
  }

Activities:
  a1 {
    duration = [3, 3]
    effects = [
      start(a1):
        machine1 -= 1:
        operators -= 2:
      end(a1):
        machine1 += 1:
        operators += 2:
    ]
  }
  a2 {
    duration = [6, 6]
    constraints = [
      (end(a2) <= 14)
    ]
    effects = [
      start(a2):
        operators -= 1:
        machine2 -= 1:
      end(a2):
        operators += 1:
        machine2 += 1:
    ]
  }
  

## Solving scheduling problems

Like all problems in the UP, we can access the `kind` field that is automatically computed to reflect the features in the problem.

In [8]:
print(problem.kind)

PROBLEM_CLASS: ['SCHEDULING']
PROBLEM_TYPE: ['SIMPLE_NUMERIC_PLANNING']
TIME: ['TIMED_EFFECTS', 'DISCRETE_TIME']
NUMBERS: ['DISCRETE_NUMBERS', 'BOUNDED_TYPES']
EFFECTS_KIND: ['DECREASE_EFFECTS', 'INCREASE_EFFECTS']
FLUENTS_TYPE: ['NUMERIC_FLUENTS']


 Currently no solver are provided in the UP that support scheduling problems natively.
 
 However, early support is provided in the development versions of the [aries](https://github.com/plaans/aries/tree/master/planning/unified/plugin) and [discrete-optimization](https://github.com/aiplan4eu/up-discreteoptimization) backends.


Reference: [Complete parser for jobshop (with operators) problems](https://github.com/aiplan4eu/unified-planning/blob/master/unified_planning/test/examples/scheduling/jobshop.py)